In [43]:
import warnings
warnings.filterwarnings("ignore")

In [44]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from optuna.logging import get_logger


import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

import seaborn as sns
import matplotlib.pyplot as plt
random_state = 6
np.random.seed(random_state)

In [45]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)


cuda


In [46]:
## look data with pandas
train_file = "data/rupturemodel_train.txt"
val_file = "data/rupturemodel_validate.txt"
test_file = "data/rupturemodel_test.txt"

df_train= pd.read_csv(train_file, sep=" ", header = None)
df_val= pd.read_csv(val_file, sep=" ", header = None)
df_test= pd.read_csv(test_file, sep=" ", header = None)

columns =  ['height', 'width', 'sxx', 'sxy', 'syy', 'sdrop', 'mud', 'dc', 'label']
df_train.columns = columns
df_val.columns = columns
df_test.columns = columns

frames = [df_train, df_val]
df_train = pd.concat(frames)
print('train data shape {} and test data shape {}'.format(np.shape(df_train), np.shape(df_test)))

train data shape (1600, 9) and test data shape (400, 9)


In [47]:
def create_new_features(df: pd.DataFrame) -> pd.DataFrame:
    df_new = df.copy()
    # Create new features
    df_new['height_width_ratio'] = df_new['height'] / df_new['width']
    df_new['normal_stress_diff'] = df_new['sxx'] - df_new['syy']
    df_new['friction_product'] = df_new['mud'] * (df_new['sdrop'])
    df_new['stress_ratio'] = df_new['sxy'] / df_new['syy']
    df_new['static_dynamic_friction_diff'] = (
        df_new['mud'] + df_new['sdrop']) - df_new['mud']
    df_new['stress_diff_dynamic_strength'] = df_new['sxy'] - \
        (df_new['syy'] * df_new['mud'])
    df_new['normalized_dc'] = df_new['dc'] / df_new['width']
    return df_new

In [48]:
X_train = df_train.drop('label', axis=1)
X_train = create_new_features(X_train)
y_train = df_train['label'].values

# Validation data
X_val = df_val.drop('label', axis=1)
X_val = create_new_features(X_val)
y_val = df_val['label'].values

# Test data
X_test = df_test.drop('label', axis=1)
X_test = create_new_features(X_test)
y_test = df_test['label'].values

In [49]:
X_train.head(5)

,height,width,sxx,sxy,syy,sdrop,mud,dc,height_width_ratio,normal_stress_diff,friction_product,stress_ratio,static_dynamic_friction_diff,stress_diff_dynamic_strength,normalized_dc
0,0.103861,1.145663,-102.509086,58.619371,-117.766562,0.483821,0.216681,0.295842,0.090656,15.257476,0.104835,-0.497759,0.483821,84.137147,0.258228
1,0.088714,1.304360,-136.062270,51.391037,-126.715571,0.345944,0.447964,0.406466,0.068013,-9.346699,0.154970,-0.405562,0.345944,108.155051,0.311621
2,0.099706,1.260377,-117.558936,40.972081,-115.529343,0.292719,0.501697,0.389360,0.079108,-2.029593,0.146856,-0.354647,0.292719,98.932806,0.308923
3,0.115749,1.191782,-128.169036,94.020712,-157.830504,0.571710,0.202831,0.408976,0.097123,29.661468,0.115961,-0.595707,0.571710,126.033631,0.343163
4,0.017900,1.108150,-106.350320,29.148969,-101.379323,0.253122,0.324653,0.398592,0.016153,-4.970997,0.082177,-0.287524,0.253122,62.062070,0.359691


### Normalize the data

In [50]:
# Normalize input data (excluding the 'label' column)
train_columns = ['height', 'width', 'sxx', 'sxy', 'syy', 'sdrop', 'mud', 'dc']
scaler = MinMaxScaler().fit(X_train[train_columns].values)
# Save scaler as pickle file using joblib
import joblib
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [51]:
# Create a PyTorch train dataset and dataloader
normalized_data = scaler.transform(X_train[train_columns].values)
X_tensor = torch.tensor(normalized_data, dtype=torch.float32)
Y_tensor = torch.tensor(y_train, dtype=torch.float32)
dataset = TensorDataset(X_tensor, Y_tensor)
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create validation dataset and dataloader
normalized_data = scaler.transform(X_val[train_columns].values)
X_tensor = torch.tensor(normalized_data, dtype=torch.float32)
Y_tensor = torch.tensor(y_val, dtype=torch.float32)
dataset = TensorDataset(X_tensor, Y_tensor)
val_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
 
 
 # Create test dataset and dataloader
normalized_data = scaler.transform(X_test[train_columns].values)
X_tensor = torch.tensor(normalized_data, dtype=torch.float32)
Y_tensor = torch.tensor(y_test, dtype=torch.float32)
dataset = TensorDataset(X_tensor, Y_tensor)
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

### Generator and Discriminator models

In [52]:
# Generator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [53]:
# Example usage
g_input_dim = 100
g_output_dim = 8

d_input_dim = 8
d_output_dim = 1

generator = Generator(g_input_dim, g_output_dim).to(device)

discriminator = Discriminator(d_input_dim, d_output_dim).to(device)

In [54]:
# Hyperparameters
epochs = 5000
batch_size = 64
lr_g = 0.0002
lr_d = 0.005
real_label_smoothing = 0.9
fake_label_smoothing = 0.1

# Loss function and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr_g)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr_d)

# Learning rate scheduler
scheduler_G = optim.lr_scheduler.StepLR(optimizer_G, step_size=200, gamma=0.5)
scheduler_D = optim.lr_scheduler.StepLR(optimizer_D, step_size=200, gamma=0.5)

# Calculate gradient penalty


def gradient_penalty(discriminator, real_data, fake_data, device='cuda'):
    batch_size = real_data.size(0)
    alpha = torch.rand(batch_size, 1).to(device)
    alpha = alpha.expand_as(real_data)
    interpolates = alpha * real_data + (1 - alpha) * fake_data
    interpolates = interpolates.requires_grad_(True)

    disc_interpolates = discriminator(interpolates)

    gradients = torch.autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                                    grad_outputs=torch.ones(
                                        disc_interpolates.size()).to(device),
                                    create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradients = gradients.view(batch_size, -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty


discriminator_losses = []
generator_losses = []

# Training loop
for epoch in range(epochs):
    for real_data, _ in train_dataloader:
        batch_size = real_data.size(0)

        # Train the discriminator
        optimizer_D.zero_grad()

        # Train with real data
        real_label = torch.ones(batch_size, 1).to(
            device) * real_label_smoothing
        real_output = discriminator(real_data.to(device))
        real_loss = criterion(real_output, real_label)

        # Train with fake data
        noise = torch.randn(batch_size, g_input_dim).to(device)
        fake_data = generator(noise)
        fake_label = torch.zeros(batch_size, 1).to(
            device) + fake_label_smoothing
        fake_output = discriminator(fake_data.detach())
        fake_loss = criterion(fake_output, fake_label)

        # Calculate gradient penalty
        gp = gradient_penalty(
            discriminator, real_data.to(device), fake_data.detach())

        # Update the discriminator
        discriminator_loss = real_loss + fake_loss + gp
        discriminator_loss.backward()
        optimizer_D.step()

        # Train the generator
        optimizer_G.zero_grad()

        # Generate fake data
        noise = torch.randn(batch_size, g_input_dim).to(device)
        fake_data = generator(noise)

        # Train the generator to fool the discriminator
        output = discriminator(fake_data)
        generator_loss = criterion(output, real_label)
        generator_loss.backward()
        optimizer_G.step()

        # Record losses
        discriminator_losses.append(discriminator_loss.item())
        generator_losses.append(generator_loss.item())

    # Save best models
    if epoch == 0:
        best_generator_loss = generator_loss.item()
        best_discriminator_loss = discriminator_loss.item()
        torch.save(generator, './models/best_generator.pth')
        torch.save(discriminator, './models/best_discriminator.pth')
    else:
        if generator_loss.item() < best_generator_loss:
            best_generator_loss = generator_loss.item()
            torch.save(generator, './models/best_generator.pth')
        if discriminator_loss.item() < best_discriminator_loss:
            best_discriminator_loss = discriminator_loss.item()
            torch.save(discriminator, './models/best_discriminator.pth')

    # Update learning rate
    scheduler_G.step()
    scheduler_D.step()

    # Print losses
    if (epoch + 1) % 100 == 0:
        print(
            f"Epoch [{epoch + 1}/{epochs}] Discriminator Loss: {discriminator_loss.item()}, Generator Loss: {generator_loss.item()}")


Epoch [100/5000] Discriminator Loss: 1.4623360633850098, Generator Loss: 0.7724428176879883
Epoch [200/5000] Discriminator Loss: 1.4954296350479126, Generator Loss: 0.7189011573791504
Epoch [300/5000] Discriminator Loss: 1.445378303527832, Generator Loss: 0.6726048588752747
Epoch [400/5000] Discriminator Loss: 1.4236160516738892, Generator Loss: 0.6961098909378052
Epoch [500/5000] Discriminator Loss: 1.4289237260818481, Generator Loss: 0.7059624791145325
Epoch [600/5000] Discriminator Loss: 1.4019849300384521, Generator Loss: 0.7089104652404785
Epoch [700/5000] Discriminator Loss: 1.3715475797653198, Generator Loss: 0.6897135972976685
Epoch [800/5000] Discriminator Loss: 1.4034706354141235, Generator Loss: 0.7038148641586304
Epoch [900/5000] Discriminator Loss: 1.3667296171188354, Generator Loss: 0.71880042552948
Epoch [1000/5000] Discriminator Loss: 1.3959250450134277, Generator Loss: 0.7300770282745361
Epoch [1100/5000] Discriminator Loss: 1.4009236097335815, Generator Loss: 0.719199

In [ ]:
# Plot losses
plt.plot(discriminator_losses, label="Discriminator Loss")
plt.plot(generator_losses, label="Generator Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend(frameon=False)
plt.show()

In [10]:
### Load generator model
generator.load_state_dict(torch.load('./models/best_generator.pt'))

<All keys matched successfully>

In [11]:
# Create a random input for the generator
random_input = torch.randn(10, g_input_dim)

# Generate data using the generator
generated_data = generator(random_input.to(device))

# Pass the generated data through the discriminator
discriminator_output = discriminator(generated_data)

In [12]:
print(f'Generated data: {generated_data}')
print(f'discriminator_output: {discriminator_output}')

Generated data: tensor([[0.7336, 0.5193, 0.1712, 0.6366, 0.0923, 0.6070, 0.5410, 0.2189],
        [0.3557, 0.9351, 0.6198, 0.9874, 0.0150, 0.0489, 0.8939, 0.1164],
        [0.7700, 0.1631, 0.5600, 0.2123, 0.5467, 0.6480, 0.4442, 0.6411],
        [0.9535, 0.0697, 0.8579, 0.1128, 0.6706, 0.9481, 0.2864, 0.6871],
        [0.7504, 0.3554, 0.5340, 0.3411, 0.3530, 0.7546, 0.3566, 0.4841],
        [0.4335, 0.4058, 0.4196, 0.5826, 0.3022, 0.3407, 0.3622, 0.5282],
        [0.8468, 0.3040, 0.5557, 0.4055, 0.3092, 0.6256, 0.3297, 0.4304],
        [0.6688, 0.4701, 0.5543, 0.2956, 0.4060, 0.6298, 0.3871, 0.4691],
        [0.3720, 0.8026, 0.8845, 0.9700, 0.0163, 0.1596, 0.6792, 0.2767],
        [0.4959, 0.6679, 0.6974, 0.6331, 0.2288, 0.1978, 0.7297, 0.4811]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
discriminator_output: tensor([[0.3854],
        [0.3919],
        [0.6415],
        [0.3651],
        [0.5171],
        [0.5084],
        [0.5156],
        [0.4269],
        [0.4707],
       

## Make predictions in supervised learning

In [13]:
def create_new_features(df: pd.DataFrame) -> pd.DataFrame:
    df_new = df.copy()
    # Create new features
    df_new['height_width_ratio'] = df_new['height'] / df_new['width']
    df_new['normal_stress_diff'] = df_new['sxx'] - df_new['syy']
    df_new['friction_product'] = df_new['mud'] * (df_new['sdrop'])
    df_new['stress_ratio'] = df_new['sxy'] / df_new['syy']
    df_new['static_dynamic_friction_diff'] = (
        df_new['mud'] + df_new['sdrop']) - df_new['mud']
    df_new['stress_diff_dynamic_strength'] = df_new['sxy'] - \
        (df_new['syy'] * df_new['mud'])
    df_new['normalized_dc'] = df_new['dc'] / df_new['width']
    return df_new


In [21]:
columns = ['height', 'width', 'sxx', 'sxy',
           'syy', 'sdrop', 'mud', 'dc']
generated_data = generated_data
# Load scaler
scaler = joblib.load('./models/scaler.pkl')

de_normalized = scaler.inverse_transform(generated_data)
df = pd.DataFrame(de_normalized, columns=columns)
df = create_new_features(df)

In [22]:
df.head()

,height,width,sxx,sxy,syy,sdrop,mud,dc,height_width_ratio,normal_stress_diff,friction_product,stress_ratio,static_dynamic_friction_diff,stress_diff_dynamic_strength,normalized_dc
0,0.143571,1.586315,-165.989685,63.009632,-146.103485,0.442983,0.401838,0.309727,0.090506,-19.886200,0.178007,-0.431267,0.442983,121.719574,0.195250
1,0.069607,2.055659,-80.442055,96.421303,-157.702148,0.220095,0.533483,0.273815,0.033861,77.260094,0.117417,-0.611414,0.220095,180.552673,0.133201
2,0.150700,1.184354,-91.846703,22.609835,-77.973572,0.459356,0.365723,0.457619,0.127242,-13.873131,0.167997,-0.289968,0.459356,51.126541,0.386387
3,0.186613,1.078868,-35.041187,13.129775,-59.404205,0.579185,0.306867,0.473755,0.172971,24.363018,0.177733,-0.221024,0.579185,31.358959,0.439122
4,0.146861,1.401411,-96.813492,34.872841,-107.022987,0.501925,0.333058,0.402627,0.104795,10.209496,0.167170,-0.325844,0.501925,70.517700,0.287301


In [23]:
### Load the model
supervised_model = lgb.Booster(model_file='./models/best_supervised_model.txt')

In [24]:
## Predicting on generated data
y_pred = supervised_model.predict(
    df, num_iteration=supervised_model.best_iteration)


In [25]:
y_pred

array([ 0.08652798,  1.00886788,  0.03809386, -0.09206561, -0.16135236,
        1.11562038, -0.03799443, -0.20279799,  1.01228269,  0.93467973])

In [26]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [27]:
y_pred

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1])

### Optimize with optuna

In [28]:
train_file = "data/rupturemodel_train.txt"
columns = ['height', 'width', 'sxx', 'sxy',
           'syy', 'sdrop', 'mud', 'dc', 'label']
df_train = pd.read_csv(train_file, sep=" ", header=None)
df_train.columns = columns

In [40]:
def objective_function(trial):
    # Suggest values for input parameters

    # Define parameter search space
    width = trial.suggest_float("width", 1.0, 2.1)
    height = trial.suggest_float("height", 0.0, 0.1 * width)
    syy = trial.suggest_float("syy", -160, -10)
    sxx_low = min(0.75 * syy, 1.25 * syy)
    sxx_high = max(0.75 * syy, 1.25 * syy)
    sxx = trial.suggest_float("sxx", sxx_low, sxx_high)
    mud = trial.suggest_float("mud", 0.2, 0.6)
    sdrop = trial.suggest_float("sdrop", 0.2, 0.8 - mud)
    random_number = trial.suggest_float("random_number", 0, 1)
    sxy = (mud + 0.02 * sdrop + random_number * 0.13 * sdrop) * syy
    dc_mean, dc_std = 0.4, 0.05
    dc = trial.suggest_float("dc", dc_mean - 3 * dc_std, dc_mean + 3 * dc_std)

    # List of all parameters
    params = [height, width, sxx, sxy, syy, sdrop, mud, dc]
    train_columns = ['height', 'width', 'sxx', 'sxy',
               'syy', 'sdrop', 'mud', 'dc']
    # Create a dataframe usinf the parameters
    df  = pd.DataFrame([params], columns=train_columns)
    
    df = create_new_features(df)
    score = supervised_model.predict(
        df, num_iteration=supervised_model.best_iteration)
    
    # Optimize for high strength and low friction coefficient
    return score

In [41]:
study = optuna.create_study( direction="maximize")
study.optimize(objective_function, n_trials=400)

[I 2023-04-21 20:12:13,444] A new study created in memory with name: no-name-9be2c026-dc60-4cf5-8109-a2696a14708e
[W 2023-04-21 20:12:13,448] Trial 0 failed with parameters: {'width': 1.7149177801488265, 'height': 0.1320691030637272, 'syy': -155.20949036302176, 'sxx': -150.41697366101653, 'mud': 0.4535752283449085, 'sdrop': 0.22511327264653958, 'random_number': 0.2202179813354791} because of the following error: TypeError("suggest_float() missing 1 required positional argument: 'high'").
Traceback (most recent call last):
  File "/home/sabber/miniconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_5047/2467038148.py", line 14, in objective_function
    sxy = trial.suggest_float(
TypeError: suggest_float() missing 1 required positional argument: 'high'
[W 2023-04-21 20:12:13,487] Trial 0 failed with value None.


TypeError: suggest_float() missing 1 required positional argument: 'high'

In [33]:
print(f"Best value: {study.best_value:.4f}")
print(f"Best parameters: {study.best_params}")

Best value: 0.7380
Best parameters: {'width': 1.854946532792431, 'height': 0.1689860140113577, 'syy': -13.844257829880108, 'sxx': -11.192579697205407, 'mud': 0.5990806399820972, 'sdrop': 0.20048905361102584, 'random_number': 0.8383799728895134, 'dc': 0.4813908860952868}


In [34]:
import optuna.visualization as vis

In [35]:
# Plot optimization history
plot_optimization_history = vis.plot_optimization_history(study)
plot_optimization_history.show()


In [36]:
# Plot parameter importances
plot_param_importances = vis.plot_param_importances(study)
plot_param_importances.show()


In [37]:
# Plot parallel coordinate plot
plot_parallel_coordinate = vis.plot_parallel_coordinate(study)
plot_parallel_coordinate.show()

In [38]:
# Plot slice plot
plot_slice = vis.plot_slice(study)
plot_slice.show()


In [42]:
plot_contour = vis.plot_contour(study, params=["mud", "height"])
plot_contour.show()

[W 2023-04-21 20:26:17,858] Your study does not have any completed trials.
